## Nama: Muhammad Zhafar Al Fathi
## NIM: 21.11.4374
## Kelas: 21-IF-08

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=5356540402a8f5650cfbe458b3ff473f7898fcc6e003ae7d5cbad4b39088d4a3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
!pip install findspark

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import py4j
print(py4j.__version__)

0.10.9.7


In [ ]:
# spark.stop()
# creating Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder\
        .appName("RDD")\
        .config('spark.ui.port', '1453')\
        .getOrCreate()

spark
sc = spark.sparkContext
sc.setCheckpointDir('checkpoint/')

In [ ]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS

In [ ]:
data = sc.textFile('/content/drive/MyDrive/File Kuliah/Tugas/Semester 5/Big Data & Predictive Analytics Lanjut/ml_ratings.txt')
ratings = data.map(lambda l: l.split('::'))

In [ ]:
for i in ratings.take(10):
  print(i)

['0', '2', '3', '1424380312']
['0', '3', '1', '1424380312']
['0', '5', '2', '1424380312']
['0', '9', '4', '1424380312']
['0', '11', '1', '1424380312']
['0', '12', '2', '1424380312']
['0', '15', '1', '1424380312']
['0', '17', '1', '1424380312']
['0', '19', '1', '1424380312']
['0', '21', '1', '1424380312']


In [ ]:
ratings_final = ratings.map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])))
from sklearn.model_selection import train_test_split
train, test = ratings_final.randomSplit([0.9, 0.1], seed=42)
train_size = train.count()
test_size = test.count()

print("Size of train set:", train_size)
print("Size of test set:", test_size)

Size of train set: 1353
Size of test set: 148


In [ ]:
def train_and_calculate_mse(train_data, test_data, rank, iterations, seed):
    # Inisialisasi model ALS
    als = ALS()

    # Melatih model ALS menggunakan data pelatihan dengan parameter yang diberikan
    model = als.train(train_data, rank=rank, iterations=iterations, seed=seed)

    # Melakukan prediksi untuk data testing
    predict = model.predictAll(test_data.map(lambda x: [x[0], x[1]]))

    # Mengubah format data untuk rates (data aktual) dan preds (prediksi) supaya bisa di-join
    rates = ratings_final.map(lambda r: ((r[0], r[1]), r[2]))
    preds = predict.map(lambda r: ((r[0], r[1]), r[2]))

    # Menggabungkan data aktual dan prediksi berdasarkan key (user, item)
    rates_and_preds = rates.join(preds)

    # Menghitung Mean Squared Error (MSE)
    MSE = rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()

    # Mengembalikan nilai MSE sebagai hasil fungsi
    return MSE


In [ ]:
mse = train_and_calculate_mse(train, test, 6, 9, seed=4374)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.9761489311059669
